In [1]:
import multiprocessing
import time
import numpy as np
import scipy as sp
import emcee
import testkit
import ciamod
import os
import gc
import sys
import pickle
from scipy.io.idl import readsav
from scipy import interpolate
from scipy.interpolate import interp1d


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
# set up the model arguments the drop these into theta(state vector) or runargs

# set up pressure grids in bar cos its intuitive
logcoarsePress = np.arange(-4.0, 2.5, 0.53)
coarsePress = pow(10,logcoarsePress)
logfinePress = np.arange(-4.0, 2.4, 0.1)
finePress = pow(10,logfinePress)
# forward model wants pressure in bar
press = finePress
#nprof = coarsePress.size
#print nprof
#print finePress.size

In [3]:
w1 = 0.7
w2 = 2.5

dist = 11.35
# hardwired FWHM of data in microns
fwhm = 0.005

npatches = 1
nclouds= 1
pcover = np.ones([npatches],dtype='f')
pcover[:] = 1.0
do_clouds = np.zeros([npatches],dtype='i')
do_clouds[:] = 1
cloudnum = np.zeros([npatches,nclouds],dtype='i')
cloudnum[:,:] = 99
cloudtype = np.array([npatches],dtype='i')
cloudtype[:] = 2

use_disort = 0 

# Set the profile type
proftype = 2


# now the linelist
# Set up number of gases, and point at the lists. see gaslist.dat
ngas = 10
gasnum = np.asfortranarray(np.array([1,4,7,8,10,11,3,2,20,21],dtype='i'))
lists = ["../Linelists/H2O_xsecs.pic","../Linelists/co_xsecs.pic","../Linelists/tio_xsecs.pic","../Linelists/vo_xsecs.pic","../Linelists/cah_xsecs.pic","../Linelists/crh_xsecs.pic" ,"../Linelists/feh_xsecs.pic","../Linelists/mgh_xsecs.pic","../Linelists/K_Mike_xsecs.pic","../Linelists/Na_Mike_xsecs.pic"]

In [4]:
# get the basic framework from water list
rawwavenum, inpress, inlinetemps, inlinelist = pickle.load( open('../Linelists/H2O_xsecs.pic', "rb" ) )
wn1 = 10000./w2
wn2 = 10000. / w1
inwavenum = np.asfortranarray(rawwavenum[np.where(np.logical_not(np.logical_or(rawwavenum[:] > wn2, rawwavenum[:] < wn1)))],dtype='float64')
ntemps = inlinetemps.size
npress= finePress.size
nwave = inwavenum.size
r1 = np.amin(np.where(np.logical_not(np.logical_or(rawwavenum[:] > wn2, rawwavenum[:] < wn1))))
r2 = np.amax(np.where(np.logical_not(np.logical_or(rawwavenum[:] > wn2, rawwavenum[:] < wn1))))

# Here we are interpolating the linelist onto our fine pressure scale.
# pickles have linelist as 4th entry....
linelist = (np.ones([ngas,npress,ntemps,nwave],order='F')).astype('float64', order='F')
for gas in range (0,ngas):
    inlinelist= pickle.load( open(lists[gas], "rb" ) )[3]
    # make missing bits crazy. They are all high pressure, low-temp points.
    #inlinelist[np.isnan(inlinelist)] = 1.0
    for i in range (0,ntemps):
        for j in range (r1,r2+1):
            pfit = interp1d(np.log10(inpress),np.log10(inlinelist[:,i,j]))
            linelist[gas,:,i,(j-r1)] = np.asfortranarray(pfit(np.log10(finePress)))

linelist[np.isnan(linelist)] = -50.0


In [5]:
# Get the cia bits
tmpcia, ciatemps = ciamod.read_cia("CIA_DS_aug_2015.dat",inwavenum)
cia = np.asfortranarray(np.empty((4,ciatemps.size,nwave)),dtype='float32')
cia[:,:,:] = tmpcia[:,:,:nwave] 
ciatemps = np.asfortranarray(ciatemps, dtype='float32')


In [6]:
# get the observed spectrum
obspec = np.asfortranarray(np.loadtxt("2M2224_mkoJcalib.dat",dtype='d',unpack='true'))



# place holder values for cloudparams
cloudparams = np.ones([5],dtype='d')
cloudparams[0] = 0.
cloudparams[1] = 10
cloudparams[2] = 12
cloudparams[3] = 1e-4
cloudparams[4] = 1e-5


runargs = dist, pcover, cloudtype,cloudparams,do_clouds,gasnum,cloudnum,inlinetemps,coarsePress,press,inwavenum,linelist,cia,ciatemps,use_disort,fwhm,obspec,proftype


In [7]:
# for proftype = 2 with no clouds, ndim = nprof + ngas-1, + (dlam,beta,logg, r2d2)
ndim =22 #(nprof + (ngas-1) + 9)
theta = np.empty(ndim)
print ndim

22


In [16]:
theta[0] = (0.5*np.random.randn()) - 3.0 # H2O
theta[1] = (0.5*np.random.randn()) - 3.0 # CO
theta[2] = (0.5*np.random.randn()) - 8.0 # TiO
theta[3] = (0.5*np.random.randn()) - 8.0 # VO
theta[4] = (1.0*np.random.randn()) - 6.0 # CrH
theta[5] = (0.5*np.random.randn()) - 4.0 # FeH
theta[6] = (0.5*np.random.randn()) - 5.0 # H2
theta[7] = (0.5*np.random.randn()) - 8.5 # CH4
theta[8] = (0.5*np.random.randn()) - 5.5 # Na+K
theta[9] = np.random.rand() + 4.2  # logg
theta[10] =   5e-20 * np.random.rand()   # r2d2
theta[11] = np.random.randn() * 0.001 # dlam
theta[12] = np.log10((np.random.rand() * (max(obspec[2,:]**2)*(10. - 0.01))) + (0.01*min(obspec[2,10::3]**2)))
#BTprof = np.loadtxt("BTtemp800_45_13.dat")
theta[13] = 1e-4 + 10. *  np.random.rand()  # P0
theta[14] = 1e-4 + np.random.rand()  # scale pressure
theta[15]  = np.random.rand()  # w0
theta[16]  = np.random.choice([-1,+1]) * np.random.rand() # gg
theta[17] = np.random.rand()  # a1
theta[18] = np.random.rand()  # a2
theta[19] =  1e-4 + (np.random.rand() * 10.) #P1
theta[20] =  theta[20] + np.random.rand() * 100.  #P2
theta[21] = 1000. + (np.random.rand() * 3000.0)  #T3

In [17]:
print theta

[ -3.20890961e+00  -2.11507731e+00  -7.65835566e+00  -8.28002468e+00
  -4.68899561e+00  -4.37621704e+00  -5.45486816e+00  -9.05273790e+00
  -4.62248446e+00   5.03140623e+00   3.11276567e-20   8.76743185e-04
  -3.02880461e+01   1.28625505e+00   1.20666865e-01   7.86554302e-01
   4.32160051e-01   6.61458629e-01   1.76932933e-01   7.13290124e-01
   1.24099771e+02   1.52244320e+03]


In [18]:
print do_clouds
print cloudnum
print cloudtype
print theta[15]
print theta[16]
r2d2= theta[10]
logg = theta[9]
D = 3.086e+16 * dist
R = np.sqrt(r2d2) * D
g = (10.**logg)/100.
M = (R**2 * g/(6.67E-11))/1.898E27
print R/70000e3, M

[1]
[[99]]
[2]
0.78655430222
0.432160050865
0.882809416661 32.4275534072


In [19]:
print testkit.lnprob(theta,*runargs) #* -2. / (obspec.shape[1] /3.)
#print testkit.lnprob(theta2,*runargs) * -2. / (obspec.shape[1] /3.)


-524.319852733


In [ ]:
len(pickle.dumps([theta,runargs], -1)) / 1e9

In [ ]:
for gas in range (0,ngas):
    test= pickle.load( open(lists[gas], "rb" ) )[3]
    test[np.isnan(test)] = 1.0
    print test[np.where(np.less(test,0))]